In [0]:
import numpy as np
import pandas as pd
from sklearn import decomposition
from scipy import linalg

# Importing comments and posts datasets

In [0]:
mental_comments = pd.read_csv('textMining.csv')
mental_comments.head()

,id,label,Category,Category old,selftext,Column1,Unnamed: 6,Unnamed: 7
0,13.0,1,Competition,Competition,A stress I always find myself undergoing is th...,NaN,rare,0.0
1,17.0,1,Competition,Competition,I've had it reviewed by a few upper years and ...,NaN,Other,0.0
2,48.0,1,Competition,Interview,Feeling stressed out. My job hunt is not going...,NaN,Working Environment,14.0
3,68.0,1,Competition,Competition,yo i feel you. a bunch of my friends are in sf...,NaN,NaN,NaN
4,301.0,1,Working Environment,Other,This is my last Monday of the co-op term. I st...,NaN,NaN,NaN


In [0]:
from sklearn.feature_extraction import stop_words
import nltk 
# nltk.download('wordnet')
# nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

import re,string

def clean_text(text):
  text = ' '.join([word for word in text.split() if word not in stopwords])
  text = ''.join([word for word in text if word not in string.punctuation])
  text = text.lower()
  return text

In [0]:
wn = nltk.WordNetLemmatizer()
def lemmatizing(text):
  text = [wn.lemmatize(word) for word in text.split()]
  return str(text)
mental_comments['non_stop'] = mental_comments['selftext'].apply(lambda x: clean_text(str(x.strip())))
mental_comments['lemm'] = mental_comments['non_stop'].apply(lambda x: lemmatizing(x))
mental_comments.head()

,id,label,Category,Category old,selftext,Column1,Unnamed: 6,Unnamed: 7,non_stop,lemm
0,13.0,1,Competition,Competition,A stress I always find myself undergoing is th...,NaN,rare,0.0,a stress i always find undergoing i feel scare...,"['a', 'stress', 'i', 'always', 'find', 'underg..."
1,17.0,1,Competition,Competition,I've had it reviewed by a few upper years and ...,NaN,Other,0.0,ive reviewed upper years friends class before ...,"['ive', 'reviewed', 'upper', 'year', 'friend',..."
2,48.0,1,Competition,Interview,Feeling stressed out. My job hunt is not going...,NaN,Working Environment,14.0,feeling stressed out my job hunt going well la...,"['feeling', 'stressed', 'out', 'my', 'job', 'h..."
3,68.0,1,Competition,Competition,yo i feel you. a bunch of my friends are in sf...,NaN,NaN,NaN,yo feel you bunch friends sf term i really str...,"['yo', 'feel', 'you', 'bunch', 'friend', 'sf',..."
4,301.0,1,Working Environment,Other,This is my last Monday of the co-op term. I st...,NaN,NaN,NaN,this last monday coop term i started really en...,"['this', 'last', 'monday', 'coop', 'term', 'i'..."


### Creating TF-IDF Matrix

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tfidf = TfidfVectorizer(stop_words='english')
vectors_tfidf = vectorizer_tfidf.fit_transform(mental_comments['lemm'])

### Applying topic modeling on top of it

In [0]:
import collections
clf = decomposition.NMF(n_components=6,random_state=1)
W1 = clf.fit_transform(vectors_tfidf)
H1 = clf.components_
vocab = np.array(vectorizer_tfidf.get_feature_names())
topics_frequency = collections.Counter(np.array(W1.argsort()[:,-1]))
topics_frequency = collections.OrderedDict(sorted(topics_frequency.items(),key= lambda kv: kv[1],reverse=True))
total_comments_count = sum(topics_frequency.values())

print('top ',6)
for k,v in topics_frequency.items():
  print('Topic {}:{}'.format(k+1, ", ".join([str(x) for x in vocab[(-H1[k]).argsort(axis=-1)[:10]]])))

top  6
Topic 1:feel, like, really, im, work, ive, people, going, friend, want
Topic 4:interview, im, ill, today, resume, midterm, got, week, said, ampx200b
Topic 2:course, learn, winter, heard, website, midterm, took, final, assignment, spring
Topic 3:term, coop, im, week, left, going, bad, fall, super, scared
Topic 6:program, hey, arbus, know, business, art, let, guy, using, offer
Topic 5:job, got, bad, coop, interviewed, stress, feeling, point, sure, like
